In [177]:
import hashlib
import struct
import time
import random
import pprint

from requests import get

import chainparams

LOCAL_WAN_IP = get('https://api.ipify.org').text
LOCAL_PORT = 8333
PEER_IP = "77.98.116.8"
PEER_PORT = 8333

# Almost all integers are encoded in little endian. Only IP or port number are encoded big endian.
# And magic bytes?


class Serializable:

    @staticmethod
    def _to_bytes(msg, length=0, byteorder='little'):
        if isinstance(msg, bytes):
            return msg
        if isinstance(msg, int):  # or isinstance(msg, bool):
            if length == 0:
                length = msg.bit_length()
            return msg.to_bytes(length, byteorder)
        elif isinstance(msg, str):
            return msg.encode(encoding='UTF-8', errors='strict')
        # TODO: add float support?
        else:
            return print("message of type %s not supported by _to_bytes()" % type(msg))


class Message(Serializable):

    def __init__(self, command, payload):
        self.magic = struct.pack('>I', chainparams.mainParams.StartString)
        self.command = command
        self.command_bytes = None
        self.length = struct.pack('<I', len(payload))
        self.payload = payload
        self.checksum = None
        self.header = None

    def serialize_payload(self):
        if not isinstance(self.payload, bytes):
            self.payload = super()._to_bytes(self.payload)
        self.length = struct.pack('<I', len(self.payload))

        double_hash = hashlib.sha256(hashlib.sha256(self.payload).digest()).digest()
        self.checksum = struct.pack('<4s', double_hash[:4])

    def generate_header(self):
        self.serialize_payload()

        # serialize and pack command message
        b = super()._to_bytes(self.command)
        self.command_bytes = struct.pack('<12s', b)

        # Create the whole header
        self.header = b"".join([self.magic, self.command_bytes, self.length, self.checksum])
        
    def to_bytes(self):
        self.generate_header()
        msg = b"".join([self.header, self.payload])
        return msg

    @staticmethod
    def to_var_int(x):
        if x < 0xFD:
            # pack as uint8_t
            return struct.pack('<B', x)
        elif x <= 0xFFFF:
            # pack as uint16_t
            return b"\xFD" + struct.pack('<H', x)
        elif x <= 0xFFFFFFFF:
            # pack as uint32_t
            return b"\xFE" + struct.pack('<I', x)
        elif x <= 0xFFFFFFFFFFFFFFFF:
            # pack as uint64_t
            return b"\xFF" + struct.pack('<Q', x)
        else:
            raise RuntimeError("integer too large for type<var_int>")

    @staticmethod
    def to_var_str(x):
        s = Serializable._to_bytes(x)
        l = len(s)
        ss = struct.pack('<%ss' % l, s)
        return Message.to_var_int(l) + ss


class NetworkAddress(Serializable):

    def __init__(self, ip, services=1, port=8333):
        self.time = struct.pack(b"<I", int(time.time()))
        self.services = struct.pack('<Q', services)

        if ':' in ip:
            self.ip = bytes(map(int, ip.split(':')))
        else:
            a = (b"\x00" * 10) + (b"\xFF" * 2)
            a_bytes = bytes(map(int, ip.split('.')))
            a += a_bytes
            self.ip = struct.pack('>16s', a)

        self.port = struct.pack('>H', port)
        self.address = b"".join([self.time, self.services, self.ip, self.port])
        self.addr_NT = b"".join([self.services, self.ip, self.port])

    def regenerate(self, _services, _ip, _port):
        self.__init__(services=_services, ip=_ip, port=_port)


class VersionMessage(Message):

    def __init__(self,
                 version,
                 services,
                 addr_recv,
                 addr_from,
                 user_agent,
                 start_height,
                 relay):

        self.version = version
        self.services = services
        self.timestamp = int(time.time())
        self.addr_recv = addr_recv
        self.addr_from = addr_from
        self.nonce = random.getrandbits(64)
        self.user_agent = user_agent
        self.start_height = start_height
        self.relay = relay
        self.payload= b""
        self.command = b"version"
        Message.__init__(self, command=self.command, payload=self.payload)

    def generate_nonce(self):
        self.nonce = random.getrandbits(64)
        return self.nonce

    def get_current_start_height(self):
        # TODO: implement this
        pass

    def to_bytes(self):
        msg = b""
        msg += struct.pack('<I', self.version)
        msg += struct.pack('<Q', self.services)
        msg += struct.pack("<I", int(time.time()))
        msg += self.addr_recv.addr_NT
        msg += self.addr_from.addr_NT
        msg += struct.pack('<Q', self.generate_nonce())
        msg += super().to_var_str(self.user_agent)
        msg += struct.pack('<I', self.start_height)
        msg += struct.pack('?', self.relay)
        self.payload = msg
        return Message.to_bytes(self)

    assigned_services = {hex(1): 'NODE_NETWORK',
                         hex(2): 'NODE_GETUTXO',
                         hex(4): 'NODE_BLOOM',
                         hex(8): 'NODE_WITNESS',
                         hex(1024): 'NODE_NETWORK_LIMITED'
                        }


class Verack(Message):

    def __init__(self):
        self.command = b"verack"
        self.payload = b""
        Message.__init__(self, command=self.command, payload=self.payload)
        
        
class Addr(Message):

    def __init__(self):
        self.command = b"addr"
        self.payload = b""
        Message.__init__(self, command=self.command, payload=self.payload)


In [179]:
addr1 = Addr()
print(vars(addr))

{'command': b'addr', 'payload': b'', 'magic': b'\xf9\xbe\xb4\xd9', 'command_bytes': None, 'length': b'\x00\x00\x00\x00', 'checksum': None, 'header': None}


In [164]:
verack = Verack()
print(verack.magic, verack.command, verack.length, verack.checksum)
verack.to_bytes()
print(verack.magic.hex(), verack.command_bytes.hex(), verack.length.hex(), verack.checksum.hex())
print(len(verack.command_bytes))

b'\xf9\xbe\xb4\xd9' b'verack' b'\x00\x00\x00\x00' None
f9beb4d9 76657261636b000000000000 00000000 5df6e0e2
12


In [165]:
magic = chainparams.mainParams.StartString.to_bytes(4, byteorder='big')
print(magic)
type(magic)

b'\xf9\xbe\xb4\xd9'


In [166]:
a = Message('block', '00000000000000001e8d6829a8a21adc5d38d0a473b144b6765798e61f98bd1d')

In [167]:
print(a._to_bytes(a.payload))

b'00000000000000001e8d6829a8a21adc5d38d0a473b144b6765798e61f98bd1d'


In [168]:
print(a.magic)
print(a.command)
print(a.length)
print(a.payload)
print("payload is type %s" % type(a.payload))
print(a.checksum)

b'\xf9\xbe\xb4\xd9'
block
b'@\x00\x00\x00'
00000000000000001e8d6829a8a21adc5d38d0a473b144b6765798e61f98bd1d
payload is type <class 'str'>
None


In [169]:
a.generate_header()

In [170]:
a.to_bytes()

b'\xf9\xbe\xb4\xd9block\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x88\x8b\xa5&00000000000000001e8d6829a8a21adc5d38d0a473b144b6765798e61f98bd1d'

In [171]:
pprint.pprint(vars(a))
print("  payload is type %s" % type(a.payload))

{'checksum': b'\x88\x8b\xa5&',
 'command': 'block',
 'command_bytes': b'block\x00\x00\x00\x00\x00\x00\x00',
 'header': b'\xf9\xbe\xb4\xd9block\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00'
           b'\x88\x8b\xa5&',
 'length': b'@\x00\x00\x00',
 'magic': b'\xf9\xbe\xb4\xd9',
 'payload': b'00000000000000001e8d6829a8a21adc5d38d0a473b144b6765798e61f98bd1d'}
  payload is type <class 'bytes'>


In [172]:
class Test(Message):
    test_number = 1000000000
    test_message = 'hello, world'
    print(test_number)
    print(Message.to_var_int(test_number))
    print(test_message)
    print(Message.to_var_str(test_message))
    


1000000000
b'\xfe\x00\xca\x9a;'
hello, world
b'\x0chello, world'


In [173]:
Message._to_bytes(1)

b'\x01'

In [174]:
n = NetworkAddress(services=1, ip="77.98.116.8", port=8333)
pprint.pprint(vars(n))

{'addr_NT': b'\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
            b'\x00\x00\xff\xffMbt\x08 \x8d',
 'address': b'c\xbc8\\\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
            b'\x00\x00\x00\x00\x00\x00\xff\xffMbt\x08 \x8d',
 'ip': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xffMbt\x08',
 'port': b' \x8d',
 'services': b'\x01\x00\x00\x00\x00\x00\x00\x00',
 'time': b'c\xbc8\\'}


In [175]:
peer = NetworkAddress(ip=PEER_IP)
local = NetworkAddress(ip=LOCAL_WAN_IP)

v = VersionMessage(version=70015,
                   services=1,
                   addr_recv=peer,
                   addr_from=local,
                   user_agent="/170000/Satoshi:0.17.0/",
                   start_height=558031,
                   relay=1)

v.to_bytes()


b'\xf9\xbe\xb4\xd9version\x00\x00\x00\x00\x00i\x00\x00\x00F\x01\x95\r\x7f\x11\x01\x00\x01\x00\x00\x00\x00\x00\x00\x00c\xbc8\\\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xffMbt\x08 \x8d\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xffm\xe0\xdb\xc4 \x8d\xb5\xe2m\rO<\xf9{\x17/170000/Satoshi:0.17.0/\xcf\x83\x08\x00\x01'

In [176]:
pprint.pprint(v.assigned_services)

{'0x1': 'NODE_NETWORK',
 '0x2': 'NODE_GETUTXO',
 '0x4': 'NODE_BLOOM',
 '0x400': 'NODE_NETWORK_LIMITED',
 '0x8': 'NODE_WITNESS'}
